In [29]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import FakeEmbeddings



In [30]:
load_dotenv()
API_KEY=os.getenv('Key')
Model="llama-3.1-70b-versatile"

In [31]:
model=ChatGroq(api_key=API_KEY, model=Model)
model.invoke('What is coding?')

AIMessage(content='Coding, also known as computer programming, is the process of designing, writing, testing, and maintaining the instructions that a computer follows to perform a specific task. These instructions, called "code," are written in one or more programming languages, which are used to communicate with a computer.\n\nThink of coding like writing a recipe for your favorite dish. Just as a recipe provides step-by-step instructions for preparing a meal, code provides instructions for a computer to execute a specific task, such as:\n\n* Calculating a math problem\n* Sorting data\n* Displaying a website\n* Playing a game\n* Controlling a robot\n\nCoding involves several steps:\n\n1. **Design**: Decide what you want the computer to do and how it will do it.\n2. **Write**: Write the code in a programming language, such as Python, Java, or C++.\n3. **Test**: Run the code to see if it works as expected.\n4. **Debug**: Fix any errors or bugs in the code.\n5. **Maintain**: Update and m

In [32]:
parser=StrOutputParser()
chain=model|parser

In [33]:
chain.invoke('What is coding?')

'Coding, also known as computer programming, is the process of designing, writing, testing, and maintaining the instructions that a computer follows to perform a specific task. These instructions, called "code," are written in one or more programming languages, which are used to communicate with a computer.\n\nThink of coding like writing a recipe for a computer. Just as a recipe provides a step-by-step guide for a human to follow, code provides a set of instructions that a computer can execute to achieve a specific goal.\n\nCoding involves several steps:\n\n1. **Design**: Define the problem or goal, and plan how to solve it.\n2. **Write**: Write the code in a programming language, such as Python, Java, or C++.\n3. **Test**: Test the code to ensure it works as expected and fix any errors.\n4. **Debug**: Identify and fix any errors or bugs in the code.\n5. **Maintain**: Update and refine the code over time to ensure it continues to work correctly.\n\nCoding is used in many areas, includ

In [34]:
from langchain.document_loaders import PyPDFLoader
# load the multiple PDF files from the directory
import glob
pdf_directory="PDF_files/*.pdf"
# find all the pdf files in the directory
pdf_files=glob.glob(pdf_directory)
# Initialize an empty list to store pages from all pdfs
all_pages=[]
# Loop over all the the PDF files and load them 
for pdf_files in pdf_files:
    # create a files loader for each pdf file
    file_loader=PyPDFLoader(pdf_files)
    # load  and split the pages
    pages=file_loader.load_and_split()
    
    #append the loaded pages to the all pages_list
    all_pages.extend(pages)
# Now all pages contain the combined images for all pdfs
print("f Total number of pages loaded :{len(all_pages)}")
    
    

f Total number of pages loaded :{len(all_pages)}


In [35]:
spliter=RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
pages=spliter.split_documents(all_pages)

In [36]:
vector_storage=FAISS.from_documents(pages, FakeEmbeddings(size=1352))
retriever=vector_storage.as_retriever()

In [37]:
question_template=""" 
your a smart bot that answers questions based on the context given to you only.
Return the answer.
context:{context}
question:{question}
"""

In [38]:
prompt=PromptTemplate.from_template(template=question_template)
print(prompt.format(context='Here is the context to use',
                    question='Answer this question based on the context'
                    ))

 
your a smart bot that answers questions based on the context given to you only.
Return the answer.
context:Here is the context to use
question:Answer this question based on the context



In [39]:
result=RunnableParallel(context=retriever, question=RunnablePassthrough())

In [40]:
chain=result|prompt|model|parser

In [41]:
retriever.invoke('Which English poet wrote A Shropshire Lad?')

[Document(metadata={'source': 'PDF_files\\who-is-who-and-what-is-what-pdf-rkl.pdf', 'page': 135}, page_content='world oldest and biggest salt mines. Salt has been mined at Khewra since 320'),
 Document(metadata={'source': 'PDF_files\\pdfcoffee.com_general-knowledge-for-pakistan-pdf-pdf-free.pdf', 'page': 38}, page_content='Registers are contained in control unit and arithmetic Logic Unit.'),
 Document(metadata={'source': 'PDF_files\\who-is-who-and-what-is-what-pdf-rkl.pdf', 'page': 44}, page_content='First century was completed by Nazar Mohammd against India in 1954 in \nLacknow. \uf020'),
 Document(metadata={'source': 'PDF_files\\who-is-who-and-what-is-what-pdf-rkl (2).pdf', 'page': 354}, page_content='Current Location: Rome.  \nMajor Highlights: Most powerful Ancient Civilization')]

In [44]:
chain.invoke('who is Nazar Mohammad')

'There is no information about Nazar Mohammad in the given context.'